In [77]:
import os
import json
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

sys.path.append('..')
import metrics as metrics
import utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['json_data_path'] = os.path.join('..', params['json_data_path'])
params['json_metadata_path'] = os.path.join('..', params['json_metadata_path'])

In [79]:
metrics.compute_average_map(params)

0.024068616137237056

In [126]:
def compute_confusion_matrix(params):
    ground_truth_json = params['json_data_path']
    prediction_json = os.path.join(params['data_dir'], 'predicted_output.json')
    metadata_path = params['json_metadata_path']

    with open(ground_truth_json, 'r') as file:
        ground_truth = json.load(file)

    with open(prediction_json, 'r') as file:
        prediction = json.load(file)

    with open(metadata_path, 'r') as file:
        metadata = json.load(file)

    return fill_confusion_matrix(prediction, ground_truth, metadata, params)

In [127]:
def calc_overlap(min_1, max_1, min_2, max_2):
    return np.max([0, np.min([max_1, max_2]) - np.max([min_1, min_2])])

In [128]:
def fill_dict(confusion_matrix,key,save_label,save_value):
    
    if key in confusion_matrix.keys():
        if save_label in confusion_matrix[key].keys():
            confusion_matrix[key][save_label] += save_value
        else:
            confusion_matrix[key][save_label] = save_value
    else:
        confusion_matrix[key] = {save_label: save_value}
    
    return confusion_matrix

In [129]:
def fill_confusion_matrix(prediction, ground_truth, metadata, params):

    confusion_matrix = dict()
    for key, value in prediction['results'].items():
        
        ground_truth_key = ground_truth['database'][key]['annotations']
        for predicted_segment in prediction['results'][key]:

            true_positive = 0
            false_positive = 0
            
            predicted_label = predicted_segment['label']
            min_pred = predicted_segment['segment'][0][0]
            max_pred = predicted_segment['segment'][0][1]
            
            for ground_truth_key_item in ground_truth_key:
                
                truth_label = ground_truth_key_item['label']
                metadata_label = metadata[truth_label]['level_2']['name']
                
                min_true = ground_truth_key_item['segment'][0]
                max_true = ground_truth_key_item['segment'][1]
                
                overlap = calc_overlap(min_pred, max_pred, min_true, max_true)
                
                if overlap != 0:
                    confusion_matrix = fill_dict(confusion_matrix,predicted_label,metadata_label,overlap)
                    
                    background = (max_pred-min_pred) - overlap
                    confusion_matrix = fill_dict(confusion_matrix,predicted_label,'background',background)
                    
                else:
                    confusion_matrix = fill_dict(confusion_matrix,predicted_label,'background',(max_pred-min_pred))

    return confusion_matrix

In [130]:
confusion_matrix = compute_confusion_matrix(params)

In [137]:
import pandas as pd
confusion_df = pd.DataFrame.from_dict(confusion_matrix, orient='index')

In [152]:
confusion_df.style

,Playing hockey,background,Participating in martial arts,"Lawn, garden, and houseplant care",Playing sports,Playing basketball,Participating in water sports,"Climbing, spelunking, caving",Playing racquet sports,Walking / exercising / playing with animals,Interior cleaning,Dancing,"Interior arrangement, decoration, & repairs",Playing musical instruments,Attending arts and entertainment,Doing gymnastics,Participating in equestrian sports,Exterior cleaning,Care for animals and pets (not veterinary care),Doing aerobics,"Washing, dressing and grooming oneself","Washing, dressing and grooming",Household & personal organization and planning,Tobacco and drug use,Playing games,Laundry,Using cardiovascular equipment,"Sewing, repairing, & maintaining textiles",Eating and Drinking,Weightlifting,Food and drink preparation,Kitchen and food clean-up,Heating and cooling
Attending arts and entertainment,nan,159.185,2.8,nan,8.4084,nan,nan,nan,nan,nan,nan,62.1876,nan,nan,17.2172,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Care for animals and pets (not veterinary care),nan,23.6142,nan,nan,nan,nan,nan,nan,nan,52.2439,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.8028,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
"Climbing, spelunking, caving",nan,90.9934,nan,nan,nan,nan,nan,83.9814,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Dancing,nan,551.455,nan,nan,38.1912,nan,2.8028,nan,nan,nan,nan,394.27,nan,nan,nan,3.36,1.23441,20.194,nan,nan,nan,nan,33.3177,nan,14.6308,nan,nan,nan,nan,nan,nan,nan,nan
Doing aerobics,nan,5.20819,6.19181,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,10.32,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Doing gymnastics,nan,11.311,nan,nan,2.8028,nan,8.51516,nan,nan,nan,nan,nan,nan,nan,5.6056,70.56,nan,nan,nan,56.0966,nan,nan,nan,nan,nan,nan,2.8,nan,nan,nan,nan,nan,nan
Exterior cleaning,nan,9.84026,nan,nan,nan,nan,nan,nan,nan,52.8384,nan,nan,nan,nan,nan,nan,nan,214.178,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
Food and drink preparation,nan,28.8255,nan,nan,2.8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2.8,nan,nan,nan,nan,nan,nan,nan,42.2052,nan,nan
Heating and cooling,nan,1.3782,nan,nan,5.8,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,1.9818
Household & personal organization and planning,nan,12.3256,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,5.6,5.8048,162.036,nan,8.4,5.8058,nan,nan,nan,nan,nan,nan,nan
